In [ ]:
# motor controll code

In [ ]:
from gpiozero import Motor
import curses

flmotor = Motor(forward=16, backward=17)
frmotor = Motor(forward=18, backward=13)
blmotor = Motor(forward=9, backward=11)
brmotor = Motor(forward=10, backward=12)

def forward():
    flmotor.forward()
    frmotor.forward()
    blmotor.forward()
    brmotor.forward()

def reverse():
    flmotor.backward()
    frmotor.backward()
    blmotor.backward()
    brmotor.backward()

def left():
    flmotor.backward()
    frmotor.forward()
    blmotor.backward()
    brmotor.forward()

def right():
    flmotor.forward()
    frmotor.backward()
    blmotor.forward()
    brmotor.backward()

def stop():
    flmotor.stop()
    frmotor.stop()
    blmotor.stop()
    brmotor.stop()

actions = {
    ord('d'): forward,
    ord('a'): reverse,
    curses.KEY_LEFT:  left,
    curses.KEY_RIGHT: right,
}

def main(window):
    next_key = None
    while True:
        curses.halfdelay(1)
        if next_key is None:
            key = window.getch()
        else:
            key = next_key
            next_key = None
        if key != -1:
            curses.halfdelay(3)
            action = actions.get(key)
            if action is not None:
                action()
            next_key = key
            while next_key == key:
                next_key = window.getch()
            stop()

curses.wrapper(main)

In [ ]:
# Pi cam

In [ ]:
pip3 install picamera

In [ ]:
import io
import picamera
import logging
import socketserver
from threading import Condition
from http import server

PAGE="""\
<html>
<head>
<title>Raspberry Pi - Video Stream</title>
</head>
<body>
<center><h1>Raspberry Pi - Video Stream</h1></center>
<center><img src="stream.mjpg" width="640" height="480"></center>
</body>
</html>
"""

class StreamingOutput(object):
    def __init__(self):
        self.frame = None
        self.buffer = io.BytesIO()
        self.condition = Condition()

    def write(self, buf):
        if buf.startswith(b'\xff\xd8'):
            self.buffer.truncate()
            with self.condition:
                self.frame = self.buffer.getvalue()
                self.condition.notify_all()
            self.buffer.seek(0)
        return self.buffer.write(buf)

class StreamingHandler(server.BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(301)
            self.send_header('Location', '/index.html')
            self.end_headers()
        elif self.path == '/index.html':
            content = PAGE.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/html')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
        elif self.path == '/stream.mjpg':
            self.send_response(200)
            self.send_header('Content-Type', 'multipart/x-mixed-replace; boundary=frame')
            self.end_headers()
            try:
                while True:
                    with output.condition:
                        output.condition.wait()
                        frame = output.frame
                    self.wfile.write(b'--frame\r\n')
                    self.send_header('Content-Type', 'image/jpeg')
                    self.send_header('Content-Length', len(frame))
                    self.end_headers()
                    self.wfile.write(frame)
                    self.wfile.write(b'\r\n')
            except Exception as e:
                logging.warning(
                    'Removed streaming client %s: %s',
                    self.client_address, str(e))
        else:
            self.send_error(404)
            self.end_headers()

class StreamingServer(socketserver.ThreadingMixIn, server.HTTPServer):
    allow_reuse_address = True
    daemon_threads = True

with picamera.PiCamera(resolution='640x480', framerate=24) as camera:
    output = StreamingOutput()
    camera.start_recording(output, format='mjpeg')
    try:
        address = ('', 8000)
        server = StreamingServer(address, StreamingHandler)
        server.serve_forever()
    finally:
        camera.stop_recording()

In [ ]:
client.py

In [ ]:
import io
import socket
import struct
import time
import picamera
import sys

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((sys.argv[1], int(sys.argv[2])))

connection = client_socket.makefile('wb')
try:
    with picamera.PiCamera() as camera:
        camera.resolution = (640, 480)
 print("starting Camera...........")
        time.sleep(2)
        stream = io.BytesIO()
        for foo in camera.capture_continuous(stream, 'jpeg'):
            connection.write(struct.pack('<L', stream.tell()))
            connection.flush()
            stream.seek(0)
            connection.write(stream.read())
            stream.seek(0)
            stream.truncate()
finally:
    connection.close()
    client_socket.close()

In [ ]:
server.py

In [ ]:
import io
import socket
import struct
from PIL import Image
import cv2
import numpy
import sys

server_socket = socket.socket()
server_socket.bind((sys.argv[1], int(sys.argv[2])))
server_socket.listen(0)
print("Listening")
connection = server_socket.accept()[0].makefile('rb')
try:
    img = None
    while True:
        image_len = struct.unpack('<L', connection.read(struct.calcsize('<L')))[0]
        if not image_len:
            break
        image_stream = io.BytesIO()
        image_stream.write(connection.read(image_len))
        image_stream.seek(0)
        image = Image.open(image_stream)
        im = cv2.cvtColor(numpy.array(image), cv2.COLOR_RGB2BGR)
        cv2.imshow('Video',im)
        if cv2.waitKey(1) & 0xFF == ord('q'):
           break
    cv2.destroyAllWindows()
finally:
    connection.close()
    server_socket.close()

In [ ]:

sudo apt-get install python-opencv
pip install numpy
pip install Pillow

yash.gps

In [ ]:
import serial
from time import sleep
import webbrowser
import sys

def GPS_info() :
  global NMEA_buff
  global lat_in_degrees
  global long_in_degrees
  nmea_time = []
  nmea_latitude = []
  nmea_longitude = []
  nmea_time = NMEA_buff[0]
  nmea_latitude = NMEA_buff[1]
  nmea_longitude = NMEA_buff[3]
  print("NMEA Time: ", nmea_time,'\n')
  print("NMEA Latitude:", nmea_latitude,"NMEA Longitude:", nmea_longitude,'\n')
  lat = float(nmea_latitude)
  longi = float(nmea_longitude)
  lat_in_degrees = convert_to_degrees(lat)
  long_in_degrees = convert_to_degrees(longi)

def convert_to_degrees(raw_value):
  decimal_value = raw_value/100.00
  degrees = int(decimal_value)
  mm_mmmm = (decimal_value - int(decimal_value))/0.6
  position = degrees + mm_mmmm
  position = "%.4f" %(position)
  return position

gpgga_info = "$GPGGA,"
ser = serial.Serial ("/dev/ttyS0")
GPGGA_buffer = 0
NMEA_buff = 0
lat_in_degrees = 0
long_in_degrees = 0
try:
  while True:
    received_data = (str)(ser.readline())
    GPGGA_data_available = received_data.find(gpgga_info)
    if (GPGGA_data_available>0):
      GPGGA_buffer = received_data.split("$GPGGA,",1)[1]
      NMEA_buff = (GPGGA_buffer.split(','))
      GPS_Info()
      print("lat in degrees:", lat_in_degrees," long in degree: ", long_in_degrees, '\n')
      map_link = 'http://maps.google.com/?q=' + lat_in_degrees + ',' + long_in_degrees
      print("<<<<<<<<press ctrl+c to plot location on google maps>>>>>>\n")
      print("------------------------------------------------------------\n")
except KeyboardInterrupt:
    webbrowser.open(map_link)
    sys.exit(0)

In [ ]:
#google map

In [ ]:
import serial
from time import sleep
import sys
import webbrowser

ser = serial.Serial ("/dev/ttyS0")
gpgga_info = "$GPGGA,"
GPGGA_buffer = 0
NMEA_buff = 0
GPGGA_data_available = ""

def convert_to_degrees(raw_value):
    decimal_value = raw_value/100.00
    degrees = int(decimal_value)
    mm_mmmm = (decimal_value – int(decimal_value))/0.6
    position = degrees + mm_mmmm
    position = "%.4f" %(position)
    return position

received_data = (str)(ser.read(200)) #read NMEA string received
GPGGA_data_available = received_data.find(gpgga_info)   #check for NMEA GPGGA string
if (GPGGA_data_available>0):
    GPGGA_buffer = received_data.split("$GPGGA,",1)[1]  #store data coming after “$GPGGA,” string
    NMEA_buff = (GPGGA_buffer.split(','))
    nmea_time = []
    nmea_latitude = []
    nmea_longitude = []
    nmea_time = NMEA_buff[0]                    #extract time from GPGGA string
    nmea_latitude = NMEA_buff[1]                #extract latitude from GPGGA string
    nmea_longitude = NMEA_buff[3]
    print("NMEA Time: ", nmea_time,'\n')
    lat = (float)(nmea_latitude)
    lat = convert_to_degrees(lat)
    longi = (float)(nmea_longitude)
    longi = convert_to_degrees(longi)
    print ("NMEA Latitude:", lat,"NMEA Longitude:" ,longi,'\n')
    map_link = 'http://maps.google.com/?q=' + lat + ','+ longi
    webbrowser.open(map_link)
sys.exit(0)

In [ ]:
#gps testing raw data
import serial
from time import sleep
import sys

ser = serial.Serial ("/dev/ttyS0")
try:
    while True:
        received_data = (str)(ser.readline()) #read NMEA string received
        print(received_data, "\n")
except KeyboardInterrupt:
    sys.exit(0)

In [ ]:
#KY036
import RPi.GPIO as GPIO

TouchPin = 13
LedPin   = 15

def setup():
 GPIO.setmode(GPIO.BOARD)       # Numbers GPIOs by physical location
 GPIO.setup(LedPin, GPIO.OUT)   # Set LedPin's mode is output
 GPIO.setup(TouchPin, GPIO.IN, pull_up_down=GPIO.PUD_UP)
 GPIO.output(LedPin, GPIO.HIGH) # Set LedPin high(+3.3V) to off led

def loop():
 while True:
  if GPIO.input(TouchPin) == GPIO.LOW:
   print '...led on'
   GPIO.output(LedPin, GPIO.LOW)  # led on
  else:
   print 'led off...'
   GPIO.output(LedPin, GPIO.HIGH) # led off

def destroy():
 GPIO.output(LedPin, GPIO.HIGH)     # led off
 GPIO.cleanup()                     # Release resource

if _name_ == '_main_':     # Program start from here
 setup()
 try:
  loop()
 except KeyboardInterrupt:  # When 'Ctrl+C' is pressed, the child program destroy() will be  executed.
  destroy()

In [ ]:
#sim28 Google
#pip install pyserial RPi.GPIO requests
import serial
import RPi.GPIO as GPIO
import requests
import time

# Configure the serial port for the GPS module
ser = serial.Serial('/dev/ttyS0', 9600, timeout=1)

# Configure the GPIO pin
GPIO.setmode(GPIO.BCM)
GPIO.setup(4, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)

def get_gps_data():
    try:
        while True:
            data = ser.readline().decode()
            if data.startswith('$GPGGA'):
                parts = data.split(',')
                if len(parts) >= 10:
                    latitude = parts[2]
                    longitude = parts[4]
                    return latitude, longitude
    except KeyboardInterrupt:
        ser.close()

def display_location(latitude, longitude):
    print(f"Latitude: {latitude}, Longitude: {longitude}")

def get_google_maps_url(latitude, longitude):
    api_key = "YOUR_GOOGLE_MAPS_API_KEY"
    return f"https://maps.googleapis.com/maps/api/staticmap?center={latitude},{longitude}&zoom=15&size=400x400&markers={latitude},{longitude}&key={api_key}"

try:
    while True:
        GPIO.wait_for_edge(4, GPIO.RISING)
        latitude, longitude = get_gps_data()
        display_location(latitude, longitude)
        maps_url = get_google_maps_url(latitude, longitude)
        print("Google Maps URL:", maps_url)
        time.sleep(5)
except KeyboardInterrupt:
    GPIO.cleanup()

In [ ]:
#lat long
import serial
import RPi.GPIO as GPIO

# Define the GPIO pin
GPIO_PIN = 17

# Initialize GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setup(GPIO_PIN, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)

# Initialize the serial port for the GPS module
ser = serial.Serial('/dev/ttyS0', 9600, timeout=1)  # Use '/dev/ttyS0' for UART on Raspberry Pi 3/4

def read_gps_data():
    try:
        while True:
            if GPIO.input(GPIO_PIN) == GPIO.HIGH:
                line = ser.readline().decode('utf-8')
                if "GPGGA" in line:
                    data = line.split(',')
                    latitude = data[2]
                    longitude = data[4]
                    print(f"Latitude: {latitude}, Longitude: {longitude}")
    except KeyboardInterrupt:
        print("GPS data read stopped.")
        ser.close()
        GPIO.cleanup()

if __name__ == "__main__":
    try:
        read_gps_data()
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        ser.close()
        GPIO.cleanup()

In [ ]:
#Only lat

import serial
import pynmea2  # You may need to install this library using pip

# Define the serial port where the GPS sensor is connected
serial_port = '/dev/ttyS0'  # Update this to the correct port on your system

# Open the serial port
ser = serial.Serial(serial_port, baudrate=9600, timeout=1)

try:
    while True:
        # Read a line from the serial port
        sentence = ser.readline().decode()

        # Check if the sentence is a valid NMEA sentence
        if sentence.startswith('$GPGGA'):
            data = pynmea2.parse(sentence)
            latitude = data.latitude
            longitude = data.longitude

            print(f'Latitude: {latitude}  Longitude: {longitude}')

except KeyboardInterrupt:
    ser.close()
    print('GPS module stopped.')